In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_children_books_final = pd.read_csv('df_children_books_final.csv')
df_children_books_final_encoded = pd.read_csv('df_children_books_final_encoded.csv')
df_interactions_final_merged = pd.read_csv('df_interactions_final_merged.csv')

/var/folders/lv/4p2_jtmx6rzdf0lvp89413tc0000gn/T/ipykernel_3975/2830472867.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_children_books_final_encoded = pd.read_csv('df_children_books_final_encoded.csv')


In [3]:
df_children_books_final.head(3)

,isbn,isbn13,book_id,title,authors,num_pages,language_code,description,format,is_ebook,popular_shelves,average_rating,ratings_count,text_reviews_count,isbn_combined,book_length,top_popular_shelves,author_ids,shelf_names
0,1599150603,9781599150604,287141,The Aeneid for Boys and Girls,"[{'author_id': '3041852', 'role': ''}]",162.0,NaN,"Relates in vigorous prose the tale of Aeneas, ...",Physical,False,"[{'count': '56', 'name': 'to-read'}, {'count':...",4.13,46,7,1599150603,medium,"[{'count': '4', 'name': 'history'}, {'count': ...",['3041852'],"['history', 'classics', 'level-4to5']"
1,1934876569,9781934876565,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"[{'author_id': '19158', 'role': ''}]",216.0,NaN,"To Kara's astonishment, she discovers that a p...",Physical,False,"[{'count': '515', 'name': 'to-read'}, {'count'...",4.22,98,6,1934876569,medium,"[{'count': '9', 'name': 'favorites'}, {'count'...",['19158'],"['favorites', 'magic', 'avalon']"
2,0590417010,9780590417013,89378,Dog Heaven,"[{'author_id': '5411', 'role': ''}]",40.0,en,In Newbery Medalist Cynthia Rylant's classic b...,Physical,False,"[{'count': '450', 'name': 'to-read'}, {'count'...",4.43,1331,193,0590417010,short,"[{'count': '8', 'name': 'pets'}, {'count': '8'...",['5411'],"['pets', 'children-s-lit', 'fiction']"


In [4]:
df_children_books_final_encoded.head(3)

,isbn,isbn13,book_id,title,authors,num_pages,description,is_ebook,popular_shelves,average_rating,...,lang_tha,lang_tur,lang_ukr,lang_vie,lang_vls,lang_yid,lang_zho,length_long,length_medium,length_short
0,1599150603,9781599150604,287141,The Aeneid for Boys and Girls,"[{'author_id': '3041852', 'role': ''}]",162.0,"Relates in vigorous prose the tale of Aeneas, ...",False,"[{'count': '56', 'name': 'to-read'}, {'count':...",4.13,...,False,False,False,False,False,False,False,False,True,False
1,1934876569,9781934876565,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"[{'author_id': '19158', 'role': ''}]",216.0,"To Kara's astonishment, she discovers that a p...",False,"[{'count': '515', 'name': 'to-read'}, {'count'...",4.22,...,False,False,False,False,False,False,False,False,True,False
2,0590417010,9780590417013,89378,Dog Heaven,"[{'author_id': '5411', 'role': ''}]",40.0,In Newbery Medalist Cynthia Rylant's classic b...,False,"[{'count': '450', 'name': 'to-read'}, {'count'...",4.43,...,False,False,False,False,False,False,False,False,False,True


In [5]:
df_interactions_final_merged.head(3)

,user_id,book_id,review_id,rating,review_text_incomplete,date_added,n_votes,review_age,processed_review,sentiment
0,8842281e1d1347389f2ab93d60773d4d,23310161,f4b4b050f4be00e9283c92a814af2670,4,Fun sequel to the original.,2015-11-17 19:37:35+00:00,7.0,3248,fun sequel original,0.680800
1,8842281e1d1347389f2ab93d60773d4d,18296097,bc9cff98f54be3b2b8c5b34598a7546c,5,NaN,2015-09-21 15:16:57+00:00,NaN,3305,NaN,0.574139
2,8842281e1d1347389f2ab93d60773d4d,817720,75fd46041466ceb406b7fd69b089b9c5,5,NaN,2015-05-21 04:29:23+00:00,NaN,3429,NaN,0.574139


In [6]:
# fill the empty rows of 'description' column with empty string so tf-idf can be applied
df_children_books_final_encoded['description'] = df_children_books_final_encoded['description'].fillna('')


In [7]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_scores =  tfidf.fit_transform(df_children_books_final_encoded['description'])
tfidf_scores.shape

(111158, 242095)

In [8]:
def cos_sim(vector_a, vector_b):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Calculate the Euclidean norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Calculate cosine similarity using the dot product and vector norms
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [11]:
def pairwise_cos_sim(vectors):
    num_vectors = vectors.shape[0]
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            # Calculate cosine similarity for each pair of vectors
            similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

    return similarity_matrix

In [12]:
tfidf_cos_sim = pairwise_cos_sim(tfidf_scores.toarray())
print(tfidf_cos_sim.shape)

: 

In [9]:
#tfidf_cosine_similarity = cosine_similarity(tfidf_scores)
#tfidf_cosine_similarity

NameError: name 'tfidf_cos_sim' is not defined